# **March Madness Machine Learning 2025** #
##### By: CoNiya Butler & Daniel Davis #####

### **Introduction** ###

This notebook explores historical NCAA Division I men's and women's basketball data to predict the outcomes of March Madness tournament games. We'll leverage provided datasets containing team information, game results, and tournament seeds to build a predictive model. Data files are prefixed with 'M' for men's, 'W' for women's,and some span both.

##### Goal: #####


Minimize the Brier score, the evaluation metric for this competition.


##### Approach: #####

1. Start with a model with the features on the list provided below.
    - Seeding differences
    - Average points Per Game
    - Win percentage regular season
    - Win percentage tournament
    - Location
    - Average scoring difference per game
2. Apply feature engineering and model tuning techniques

### **Import Libraries** ###

In [115]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import log_loss, brier_score_loss, mean_squared_error, roc_curve, auc
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, accuracy_score
import warnings
warnings.filterwarnings("ignore")

### **Load Data** ###

Load the datasets need for the model M is for men's basketball and W is for women's basketball

In [132]:
M_seed_df = pd.read_csv("/home/pysparkdevcontainer/code/data/MNCAATourneySeeds.csv")
W_seed_df = pd.read_csv("/home/pysparkdevcontainer/code/data/WNCAATourneySeeds.csv")
M_regular_results = pd.read_csv("/home/pysparkdevcontainer/code/data/MRegularSeasonDetailedResults.csv")
M_tourney_results = pd.read_csv("/home/pysparkdevcontainer/code/data/MNCAATourneyDetailedResults.csv")
W_regular_results = pd.read_csv("/home/pysparkdevcontainer/code/data/WRegularSeasonDetailedResults.csv")
W_tourney_results = pd.read_csv("/home/pysparkdevcontainer/code/data/WNCAATourneyDetailedResults.csv")
submissionstage2_df = pd.read_csv("/home/pysparkdevcontainer/code/data/SampleSubmissionStage2.csv")
submissionstage1_df = pd.read_csv("/home/pysparkdevcontainer/code/data/SampleSubmissionStage1.csv")

### **Data Preprocessing** ###

In [133]:
# Join the men's and women's dataset together
regular_results = pd.concat([M_regular_results, W_regular_results])
tourney_results = pd.concat([M_tourney_results, W_tourney_results])
seed_df = pd.concat([M_seed_df, W_seed_df])
submission_df = pd.concat([submissionstage1_df, submissionstage2_df])

In [135]:
#This function will caculate the teams' average win score, loss score, points and point differential, for each season.
def compute_stats(results):
    wins = results.groupby(["Season", "WTeamID"])["WScore"].agg(["count", "mean"]).rename(columns={"count": "Wins", "mean": "AvgWinScore"}).reset_index()
    wins = wins.rename(columns={'WTeamID': 'Team1'})
    losses = results.groupby(["Season", "LTeamID"])["LScore"].agg(["count", "mean"]).rename(columns={"count": "Losses", "mean": "AvgLossScore"}).reset_index()
    losses = losses.rename(columns={'LTeamID' : 'Team1'})

    stats = wins.merge(losses, on=["Season", "Team1"]).fillna(0)
    stats["TotalGames"] = stats["Wins"] + stats["Losses"]
    stats["WinRate"] = stats["Wins"] / stats["TotalGames"]
    
   
    stats["AvgScore"] = (stats["Wins"] * stats["AvgWinScore"] + stats["Losses"] * stats["AvgLossScore"]) / stats["TotalGames"]
    

    stats["AvgScoreDiff"] = stats["AvgWinScore"] - stats["AvgLossScore"]
    
    return stats

season_stats = compute_stats(regular_results)
tourney_stats = compute_stats(tourney_results)

In [136]:
season_stats

,Season,Team1,Wins,AvgWinScore,Losses,AvgLossScore,TotalGames,WinRate,AvgScore,AvgScoreDiff
0,2003,1102,12,68.750000,16,48.625000,28,0.428571,57.250000,20.125000
1,2003,1103,13,87.769231,14,70.428571,27,0.481481,78.777778,17.340659
2,2003,1104,17,74.705882,11,60.909091,28,0.607143,69.285714,13.796791
3,2003,1105,7,79.428571,19,68.947368,26,0.269231,71.769231,10.481203
4,2003,1106,13,68.307692,15,59.533333,28,0.464286,63.607143,8.774359
...,...,...,...,...,...,...,...,...,...,...
13531,2025,3476,11,70.818182,12,57.000000,23,0.478261,63.608696,13.818182
13532,2025,3477,4,72.250000,18,59.666667,22,0.181818,61.954545,12.583333
13533,2025,3478,4,67.000000,20,49.300000,24,0.166667,52.250000,17.700000
13534,2025,3479,6,71.500000,15,58.733333,21,0.285714,62.380952,12.766667


In [143]:
def feature_selection(results_df, df):
    df['Season'] = df['ID'].map(lambda x: x.split('_')[0]).astype(int)
    df['Team1'] = df['ID'].map(lambda x: x.split('_')[1]).astype(int)
    df['Team2'] = df['ID'].map(lambda x: x.split('_')[2]).astype(int)

    feature_df = df.merge(results_df[['Season', 'Team1' , 'WinRate', 'AvgScore', 'AvgScoreDiff']], on=['Season', 'Team1'])
    feature_df = feature_df.rename(columns={'WinRate': 'Team1WinRatio', 'AvgScore': 'Team1AvgScore', 'AvgScoreDiff': 'Team1AvgScoreDiff'})
    results_df = results_df.rename(columns={'Team1': 'Team2'})
    feature_df = feature_df.merge(results_df[['Season', 'Team2' , 'WinRate', 'AvgScore', 'AvgScoreDiff']], on=['Season', 'Team2'])
    feature_df = feature_df.rename(columns={'WinRate': 'Team2WinRatio', 'AvgScore': 'Team2AvgScore', 'AvgScoreDiff': 'Team2AvgScoreDiff'})
    feature_df['Team1WinPrediction'] = 1 / (1 + 10 ** ((feature_df['Team2WinRatio']*1000 - feature_df['Team1WinRatio']*1000) / 400))
    
    feature_df = feature_df.drop(['ID','Pred'], axis=1)
    return feature_df


In [145]:
season_training_df = feature_selection(season_stats, submission_df)
tourney_training_df = feature_selection(tourney_stats, submission_df)
tourney_training_df['Season\Tourney'] = 1
season_training_df['Season\Tourney'] = 0

In [147]:
training_df = pd.concat([season_training_df, tourney_training_df])
training_df

,Season,Team1,Team2,Team1WinRatio,Team1AvgScore,Team1AvgScoreDiff,Team2WinRatio,Team2AvgScore,Team2AvgScoreDiff,Team1WinPrediction,Season\Tourney
0,2021,1101,1102,0.826087,76.304348,15.197368,0.200000,58.720000,8.350000,0.973508,0
1,2021,1101,1103,0.826087,76.304348,15.197368,0.619048,76.571429,8.596154,0.767065,0
2,2021,1101,1104,0.826087,76.304348,15.197368,0.800000,79.566667,19.666667,0.537472,0
3,2021,1101,1105,0.826087,76.304348,15.197368,0.400000,63.333333,21.666667,0.920763,0
4,2021,1101,1106,0.826087,76.304348,15.197368,0.222222,64.222222,5.821429,0.970001,0
...,...,...,...,...,...,...,...,...,...,...,...
4422,2024,3428,3439,0.500000,67.000000,2.000000,0.500000,82.000000,20.000000,0.500000,1
4423,2024,3428,3452,0.500000,67.000000,2.000000,0.500000,58.500000,9.000000,0.500000,1
4424,2024,3435,3439,0.500000,67.500000,9.000000,0.500000,82.000000,20.000000,0.500000,1
4425,2024,3435,3452,0.500000,67.500000,9.000000,0.500000,58.500000,9.000000,0.500000,1


In [ ]:
training_df[(training_df['Season'] == 2021) & (training_df['Season\Tourney'] == 1)]

Final_training_df = training_df.query('Season >= 2023')
Final_training_df

,Season,Team1,Team2,Team1WinRatio,Team1AvgScore,Team1AvgScoreDiff,Team2WinRatio,Team2AvgScore,Team2AvgScoreDiff,Team1WinPrediction,Season\Tourney
240956,2023,1101,1102,0.346154,71.192308,11.601307,0.437500,66.937500,14.460317,0.371490,0
240957,2023,1101,1103,0.346154,71.192308,11.601307,0.645161,74.161290,13.495455,0.151713,0
240958,2023,1101,1104,0.346154,71.192308,11.601307,0.852941,82.176471,15.213793,0.051305,0
240959,2023,1101,1105,0.346154,71.192308,11.601307,0.400000,67.966667,4.638889,0.423124,0
240960,2023,1101,1106,0.346154,71.192308,11.601307,0.233333,62.433333,9.683230,0.656886,0
...,...,...,...,...,...,...,...,...,...,...,...
4422,2024,3428,3439,0.500000,67.000000,2.000000,0.500000,82.000000,20.000000,0.500000,1
4423,2024,3428,3452,0.500000,67.000000,2.000000,0.500000,58.500000,9.000000,0.500000,1
4424,2024,3435,3439,0.500000,67.500000,9.000000,0.500000,82.000000,20.000000,0.500000,1
4425,2024,3435,3452,0.500000,67.500000,9.000000,0.500000,58.500000,9.000000,0.500000,1


### **Exploratory Data Analysis (EDA)** ###

In [157]:
Final_training_df.columns

Index(['Season', 'Team1', 'Team2', 'Team1WinRatio', 'Team1AvgScore',
       'Team1AvgScoreDiff', 'Team2WinRatio', 'Team2AvgScore',
       'Team2AvgScoreDiff', 'Team1WinPrediction', 'Season\Tourney'],
      dtype='object')

### **Model Training** ###

In [188]:
# Split features and target
X = training_df[['Season', 'Team1', 'Team2', 'Team1WinRatio', 'Team1AvgScore', 'Team1AvgScoreDiff', 'Team2WinRatio', 'Team2AvgScore', 'Team2AvgScoreDiff', 'Season\Tourney']]
y = training_df["Team1WinPrediction"]

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the numerical features (excluding TeamID)
scaler = StandardScaler()
X_train.iloc[:, 3:] = scaler.fit_transform(X_train.iloc[:, 3:])
X_test.iloc[:, 3:] = scaler.transform(X_test.iloc[:, 3:])

# Convert to DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define model parameters
params = {
    "objective": "binary:logistic",  # Probability prediction
    "eval_metric": "logloss",
    "eta": 0.1,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "seed": 42
}

# Train model
num_rounds = 100
bst = xgb.train(params, dtrain, num_rounds)

# Predict probabilities
y_pred_proba = bst.predict(dtest)

print("Example Predictions:", y_pred_proba[:5])

Example Predictions: [0.35652795 0.45687845 0.23934785 0.912854   0.7322806 ]


### **Prediction Submission** ###

In [176]:
submissionstage2_df['Season'] = submissionstage2_df['ID'].map(lambda x: x.split('_')[0]).astype(int)
submissionstage2_df['Team1'] = submissionstage2_df['ID'].map(lambda x: x.split('_')[1]).astype(int)
submissionstage2_df['Team2'] = submissionstage2_df['ID'].map(lambda x: x.split('_')[2]).astype(int)
submissionstage2_df

,ID,Pred,Season,Team1,Team2
0,2025_1101_1102,0.5,2025,1101,1102
1,2025_1101_1103,0.5,2025,1101,1103
2,2025_1101_1104,0.5,2025,1101,1104
3,2025_1101_1105,0.5,2025,1101,1105
4,2025_1101_1106,0.5,2025,1101,1106
...,...,...,...,...,...
131402,2025_3477_3479,0.5,2025,3477,3479
131403,2025_3477_3480,0.5,2025,3477,3480
131404,2025_3478_3479,0.5,2025,3478,3479
131405,2025_3478_3480,0.5,2025,3478,3480


,ID,Pred,Season,Team1,Team2
0,2025_1101_1102,0.5,2025,1101,1102
1,2025_1101_1103,0.5,2025,1101,1103
2,2025_1101_1104,0.5,2025,1101,1104
3,2025_1101_1105,0.5,2025,1101,1105
4,2025_1101_1106,0.5,2025,1101,1106
...,...,...,...,...,...
131402,2025_3477_3479,0.5,2025,3477,3479
131403,2025_3477_3480,0.5,2025,3477,3480
131404,2025_3478_3479,0.5,2025,3478,3479
131405,2025_3478_3480,0.5,2025,3478,3480


In [ ]:
season_training_df
total_stat_df = pd.concat([season_training_df, tourney_training_df])

,Season,Team1,Team2,Team1WinRatio,Team1AvgScore,Team1AvgScoreDiff,Team2WinRatio,Team2AvgScore,Team2AvgScoreDiff,Team1WinPrediction,Season\Tourney
0,2021,1101,1102,0.826087,76.304348,15.197368,0.200000,58.720000,8.350000,0.973508,0
1,2021,1101,1103,0.826087,76.304348,15.197368,0.619048,76.571429,8.596154,0.767065,0
2,2021,1101,1104,0.826087,76.304348,15.197368,0.800000,79.566667,19.666667,0.537472,0
3,2021,1101,1105,0.826087,76.304348,15.197368,0.400000,63.333333,21.666667,0.920763,0
4,2021,1101,1106,0.826087,76.304348,15.197368,0.222222,64.222222,5.821429,0.970001,0
...,...,...,...,...,...,...,...,...,...,...,...
4422,2024,3428,3439,0.500000,67.000000,2.000000,0.500000,82.000000,20.000000,0.500000,1
4423,2024,3428,3452,0.500000,67.000000,2.000000,0.500000,58.500000,9.000000,0.500000,1
4424,2024,3435,3439,0.500000,67.500000,9.000000,0.500000,82.000000,20.000000,0.500000,1
4425,2024,3435,3452,0.500000,67.500000,9.000000,0.500000,58.500000,9.000000,0.500000,1


In [174]:
# Prediction dataset
pred = total_stat_df[(total_stat_df['Season'] == 2025) | (total_stat_df['Season'] == 2024)].groupby(['Team1', "Team2"]).mean().reset_index()
pred['Season'] = 2025
pred

,Team1,Team2,Season,Team1WinRatio,Team1AvgScore,Team1AvgScoreDiff,Team2WinRatio,Team2AvgScore,Team2AvgScoreDiff,Team1WinPrediction,Season\Tourney
0,1101,1102,2025,0.430352,69.052053,12.218487,0.202854,64.253722,12.486495,0.780540,0.0
1,1101,1103,2025,0.430352,69.052053,12.218487,0.739583,77.838542,12.586842,0.152066,0.0
2,1101,1104,2025,0.430352,69.052053,12.218487,0.748125,90.535000,15.010823,0.156325,0.0
3,1101,1105,2025,0.430352,69.052053,12.218487,0.297101,70.283267,9.754902,0.682589,0.0
4,1101,1106,2025,0.430352,69.052053,12.218487,0.392414,69.574483,9.331203,0.552324,0.0
...,...,...,...,...,...,...,...,...,...,...,...
131388,3477,3479,2025,0.181818,61.954545,12.583333,0.285714,62.380952,12.766667,0.354784,0.0
131389,3477,3480,2025,0.181818,61.954545,12.583333,0.391304,67.043478,1.206349,0.230427,0.0
131390,3478,3479,2025,0.166667,52.250000,17.700000,0.285714,62.380952,12.766667,0.335081,0.0
131391,3478,3480,2025,0.166667,52.250000,17.700000,0.391304,67.043478,1.206349,0.215325,0.0


In [177]:
final_pred_df = pd.merge(submissionstage2_df, pred, on=['Season', 'Team1', 'Team2'], how='left')

In [178]:
final_pred_df

,ID,Pred,Season,Team1,Team2,Team1WinRatio,Team1AvgScore,Team1AvgScoreDiff,Team2WinRatio,Team2AvgScore,Team2AvgScoreDiff,Team1WinPrediction,Season\Tourney
0,2025_1101_1102,0.5,2025,1101,1102,0.430352,69.052053,12.218487,0.202854,64.253722,12.486495,0.780540,0.0
1,2025_1101_1103,0.5,2025,1101,1103,0.430352,69.052053,12.218487,0.739583,77.838542,12.586842,0.152066,0.0
2,2025_1101_1104,0.5,2025,1101,1104,0.430352,69.052053,12.218487,0.748125,90.535000,15.010823,0.156325,0.0
3,2025_1101_1105,0.5,2025,1101,1105,0.430352,69.052053,12.218487,0.297101,70.283267,9.754902,0.682589,0.0
4,2025_1101_1106,0.5,2025,1101,1106,0.430352,69.052053,12.218487,0.392414,69.574483,9.331203,0.552324,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131402,2025_3477_3479,0.5,2025,3477,3479,0.181818,61.954545,12.583333,0.285714,62.380952,12.766667,0.354784,0.0
131403,2025_3477_3480,0.5,2025,3477,3480,0.181818,61.954545,12.583333,0.391304,67.043478,1.206349,0.230427,0.0
131404,2025_3478_3479,0.5,2025,3478,3479,0.166667,52.250000,17.700000,0.285714,62.380952,12.766667,0.335081,0.0
131405,2025_3478_3480,0.5,2025,3478,3480,0.166667,52.250000,17.700000,0.391304,67.043478,1.206349,0.215325,0.0


In [183]:
final_pred_df.fillna(0)

,ID,Pred,Season,Team1,Team2,Team1WinRatio,Team1AvgScore,Team1AvgScoreDiff,Team2WinRatio,Team2AvgScore,Team2AvgScoreDiff,Team1WinPrediction,Season\Tourney
0,2025_1101_1102,0.5,2025,1101,1102,0.430352,69.052053,12.218487,0.202854,64.253722,12.486495,0.780540,0.0
1,2025_1101_1103,0.5,2025,1101,1103,0.430352,69.052053,12.218487,0.739583,77.838542,12.586842,0.152066,0.0
2,2025_1101_1104,0.5,2025,1101,1104,0.430352,69.052053,12.218487,0.748125,90.535000,15.010823,0.156325,0.0
3,2025_1101_1105,0.5,2025,1101,1105,0.430352,69.052053,12.218487,0.297101,70.283267,9.754902,0.682589,0.0
4,2025_1101_1106,0.5,2025,1101,1106,0.430352,69.052053,12.218487,0.392414,69.574483,9.331203,0.552324,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131402,2025_3477_3479,0.5,2025,3477,3479,0.181818,61.954545,12.583333,0.285714,62.380952,12.766667,0.354784,0.0
131403,2025_3477_3480,0.5,2025,3477,3480,0.181818,61.954545,12.583333,0.391304,67.043478,1.206349,0.230427,0.0
131404,2025_3478_3479,0.5,2025,3478,3479,0.166667,52.250000,17.700000,0.285714,62.380952,12.766667,0.335081,0.0
131405,2025_3478_3480,0.5,2025,3478,3480,0.166667,52.250000,17.700000,0.391304,67.043478,1.206349,0.215325,0.0


In [185]:
final_pred_df['Season\Tourney'] = 1
final_pred_df

,ID,Pred,Season,Team1,Team2,Team1WinRatio,Team1AvgScore,Team1AvgScoreDiff,Team2WinRatio,Team2AvgScore,Team2AvgScoreDiff,Team1WinPrediction,Season\Tourney
0,2025_1101_1102,0.5,2025,1101,1102,0.430352,69.052053,12.218487,0.202854,64.253722,12.486495,0.780540,1
1,2025_1101_1103,0.5,2025,1101,1103,0.430352,69.052053,12.218487,0.739583,77.838542,12.586842,0.152066,1
2,2025_1101_1104,0.5,2025,1101,1104,0.430352,69.052053,12.218487,0.748125,90.535000,15.010823,0.156325,1
3,2025_1101_1105,0.5,2025,1101,1105,0.430352,69.052053,12.218487,0.297101,70.283267,9.754902,0.682589,1
4,2025_1101_1106,0.5,2025,1101,1106,0.430352,69.052053,12.218487,0.392414,69.574483,9.331203,0.552324,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131402,2025_3477_3479,0.5,2025,3477,3479,0.181818,61.954545,12.583333,0.285714,62.380952,12.766667,0.354784,1
131403,2025_3477_3480,0.5,2025,3477,3480,0.181818,61.954545,12.583333,0.391304,67.043478,1.206349,0.230427,1
131404,2025_3478_3479,0.5,2025,3478,3479,0.166667,52.250000,17.700000,0.285714,62.380952,12.766667,0.335081,1
131405,2025_3478_3480,0.5,2025,3478,3480,0.166667,52.250000,17.700000,0.391304,67.043478,1.206349,0.215325,1


In [186]:
final_pred_df.drop(columns=['ID', 'Pred', 'Team1WinPrediction'])

,Season,Team1,Team2,Team1WinRatio,Team1AvgScore,Team1AvgScoreDiff,Team2WinRatio,Team2AvgScore,Team2AvgScoreDiff,Season\Tourney
0,2025,1101,1102,0.430352,69.052053,12.218487,0.202854,64.253722,12.486495,1
1,2025,1101,1103,0.430352,69.052053,12.218487,0.739583,77.838542,12.586842,1
2,2025,1101,1104,0.430352,69.052053,12.218487,0.748125,90.535000,15.010823,1
3,2025,1101,1105,0.430352,69.052053,12.218487,0.297101,70.283267,9.754902,1
4,2025,1101,1106,0.430352,69.052053,12.218487,0.392414,69.574483,9.331203,1
...,...,...,...,...,...,...,...,...,...,...
131402,2025,3477,3479,0.181818,61.954545,12.583333,0.285714,62.380952,12.766667,1
131403,2025,3477,3480,0.181818,61.954545,12.583333,0.391304,67.043478,1.206349,1
131404,2025,3478,3479,0.166667,52.250000,17.700000,0.285714,62.380952,12.766667,1
131405,2025,3478,3480,0.166667,52.250000,17.700000,0.391304,67.043478,1.206349,1


In [190]:
# Split features and target
X = final_pred_df[['Season', 'Team1', 'Team2', 'Team1WinRatio', 'Team1AvgScore', 'Team1AvgScoreDiff', 'Team2WinRatio', 'Team2AvgScore', 'Team2AvgScoreDiff', 'Season\Tourney']]

# Scale the numerical features (excluding TeamID)
scaler = StandardScaler()
X.iloc[:, 3:] = scaler.fit_transform(X.iloc[:, 3:])

dnew = xgb.DMatrix(X.values)

In [191]:
y_new_pred_proba = bst.predict(dnew)

# Predict probabilities
final_pred_df["Team1_Win_Probability"] = bst.predict(dnew)  # Convert to %

final_pred_df

ValueError: training data did not have the following fields: Season, Team1, Team2, Team1WinRatio, Team1AvgScore, Team1AvgScoreDiff, Team2WinRatio, Team2AvgScore, Team2AvgScoreDiff, Season\Tourney